In [12]:
from datasets import load_dataset
import pandas as pd
import os
import requests
from tqdm import tqdm


In [3]:
dataset = load_dataset("DBQ/Chanel.Product.prices.Germany")
dataset

Generating train split: 100%|██████████| 1428/1428 [00:00<00:00, 50764.64 examples/s]


DatasetDict({
    train: Dataset({
        features: ['website_name', 'competence_date', 'country_code', 'currency_code', 'brand', 'category1_code', 'category2_code', 'category3_code', 'product_code', 'title', 'itemurl', 'imageurl', 'full_price', 'price', 'full_price_eur', 'price_eur', 'flg_discount'],
        num_rows: 1428
    })
})

In [4]:
df = dataset["train"].to_pandas()
df.head()


,website_name,competence_date,country_code,currency_code,brand,category1_code,category2_code,category3_code,product_code,title,itemurl,imageurl,full_price,price,full_price_eur,price_eur,flg_discount
0,Chanel,2023-11-17,DEU,EUR,CHANEL,FASHION,HANDBAGS,N.A.,PAS4152B13455,Tweed & goldfarbenes Metall Schwarz Rosa & B...,https://www.chanel.com/de/mode/p/AS4152B13455N...,https://res.cloudinary.com/dwayrkp5z/image/fet...,5200.0,5200.0,5200.0,5200.0,0
1,Chanel,2023-11-17,DEU,EUR,CHANEL,FASHION,HANDBAGS,N.A.,PA67085Y09953,n.a.,https://www.chanel.com/de/mode/p/A67085Y099539...,https://res.cloudinary.com/dwayrkp5z/image/fet...,5750.0,5750.0,5750.0,5750.0,0
2,Chanel,2023-11-17,DEU,EUR,CHANEL,EYEWEAR,BLUE LIGHT GLASSES,N.A.,PA71441,Schwarz,https://www.chanel.com/de/brillen/p/A71441X081...,https://res.cloudinary.com/dwayrkp5z/image/fet...,420.0,420.0,420.0,420.0,0
3,Chanel,2023-11-17,DEU,EUR,CHANEL,EYEWEAR,SUNGLASSES,N.A.,PA71449,Silberfarben. Gläser: Braun Mit Verlauf,https://www.chanel.com/de/brillen/p/A71449X099...,https://res.cloudinary.com/dwayrkp5z/image/fet...,580.0,580.0,580.0,580.0,0
4,Chanel,2023-11-17,DEU,EUR,CHANEL,EYEWEAR,SUNGLASSES,N.A.,PA71466,Blau & Rosa. Gläser: Grau Mit Verlauf,https://www.chanel.com/de/brillen/p/A71466X010...,https://res.cloudinary.com/dwayrkp5z/image/fet...,480.0,480.0,480.0,480.0,0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1428 entries, 0 to 1427
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   website_name     1428 non-null   object 
 1   competence_date  1428 non-null   object 
 2   country_code     1428 non-null   object 
 3   currency_code    1428 non-null   object 
 4   brand            1428 non-null   object 
 5   category1_code   1428 non-null   object 
 6   category2_code   1428 non-null   object 
 7   category3_code   1428 non-null   object 
 8   product_code     1428 non-null   object 
 9   title            1428 non-null   object 
 10  itemurl          1428 non-null   object 
 11  imageurl         1428 non-null   object 
 12  full_price       1428 non-null   float64
 13  price            1428 non-null   float64
 14  full_price_eur   1428 non-null   float64
 15  price_eur        1428 non-null   float64
 16  flg_discount     1428 non-null   int64  
dtypes: float64(4),

In [16]:

url_column = "imageurl"      # 🔁 change le nom ici

# 2. Créer un dossier pour stocker les images
output_dir = "Images"
os.makedirs(output_dir, exist_ok=True)

valid_rows = []  # stocke les lignes valides
image_paths = [] # stocke les chemins d'images valides

# 3. Télécharger chaque image
for idx, url in tqdm(df[url_column].items(), total=len(df), desc="Téléchargement images"):

    if pd.isna(url):
        continue

    filename = os.path.join(output_dir, f"{idx:05d}.jpg")

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        # Sauvegarde de l'image
        with open(filename, "wb") as f:
            f.write(response.content)

        # Ligne valide → on stocke la ligne + chemin
        valid_rows.append(idx)
        image_paths.append(filename)

    except:
        # Pas d'affichage d'erreur (totalement silencieux)
        pass

# 4. Nettoyage : garder uniquement les bonnes lignes
df_clean = df.loc[valid_rows].reset_index(drop=True)

# Ajout de la colonne image_path
df_clean["image_path"] = image_paths

# 5. Sauvegarde du dataset propre
df_clean.to_csv("data_clean.csv", index=False)

print("✔ Téléchargement terminé")
print(f"✔ Lignes valides conservées : {len(df_clean)}")
print("✔ Chemins d'images ajoutés dans la colonne 'image_path'")
print("✔ Dataset nettoyé : data_clean.csv")


Téléchargement images: 100%|██████████| 1428/1428 [06:22<00:00,  3.73it/s]

✔ Téléchargement terminé
✔ Lignes valides conservées : 900
✔ Chemins d'images ajoutés dans la colonne 'image_path'
✔ Dataset nettoyé : data_clean.csv
